In [8]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

In [1]:
DATASETS = [
    "SocioFactors_Processed.csv",
    # Add another file/delete a file if there's any change in dataset HERE
    "CPI_Food_Processed.csv",
    "Net_FDI_Processed.csv",
    "Prevalence_of_Undernourishment_Processed.csv"
]

In [2]:
# Merge all csv files
files = [".\datasets\Processed\%s" % x for x in DATASETS]
try:    
    dfs = [pd.read_csv(file, encoding='utf8') for file in files]
except:
    dfs = [pd.read_csv(file, encoding='ISO-8859-1') for file in files]

# Drop columns
for i in range(1, len(dfs)):
    columns = ["Unnamed: 0", "Country_Name", "ISO3 Code", "IS03_Code"]
    for col in columns:
        if col in dfs[i].columns:
            dfs[i].drop(columns=[col], inplace=True)
    if "Area Code (M49)" in dfs[i].columns:
        dfs[i].rename(columns={"Area Code (M49)": "M49_Code"}, inplace=True)
    if "Net_FDI" in DATASETS[i]:
        dfs[i].drop(columns=dfs[i].columns[dfs[i].columns.str.contains("Total_FDI")], inplace=True)
for i in range(len(dfs)):
    print(dfs[i].columns)

# Merge
df = dfs[0]
for i in range(1, len(dfs)):
    df = df.merge(dfs[i], on="M49_Code", how="inner")
df

NameError: name 'pd' is not defined

## Preprocessing

In [11]:
# Replace all null values by mean
df.fillna(df.mean(), inplace=True)
df.isnull().sum()

C:\Users\admin\AppData\Local\Temp\ipykernel_44292\1916672156.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


M49_Code                               0
Country_Name                           0
ISO3 Code                              0
CPI_Food_2016                          0
CPI_Food_2017                          0
CPI_Food_2018                          0
CPI_Food_2019                          0
CPI_Food_2020                          0
CPI_Food_Avg                           0
Net_FDI_2016                           0
Net_FDI_2017                           0
Net_FDI_2018                           0
Net_FDI_2019                           0
Net_FDI_2020                           0
ISO3_Code                              0
Life_Expectancy_2016                   0
Life_Expectancy_2017                   0
Life_Expectancy_2018                   0
Life_Expectancy_2019                   0
Life_Expectancy_2020                   0
Life_Expectancy_Avg                    0
Mean_Years_Of_Schooling_2016           0
Mean_Years_Of_Schooling_2017           0
Mean_Years_Of_Schooling_2018           0
Mean_Years_Of_Sc

## Convert to excel format

In [12]:
# Create dataframes corr. to each year
import csv

df_exc = df.copy()
df_exc = df_exc.loc[:, ~df_exc.columns.str.contains('Avg')]
col_names = df_exc.columns[:3].tolist() + ["Year"] + [x.replace("_2020", "") for x in df_exc.columns[df_exc.columns.str.contains('_2020')].values]
print(col_names)
n = np.count_nonzero(df_exc.columns.str.contains('_2020')) + 4
years = [2016, 2017, 2018, 2019, 2020]
dfs = [None, None, None, None, None]
data = []
for i, year in enumerate(years):
    df_info = df_exc.iloc[:, 0:3]
    df_info["Year"] = year
    dfs[i] = pd.concat([df_info, df_exc.loc[:, df_exc.columns.str.contains(str(year))]], axis=1)
    assert dfs[i].shape[1] == n
    data += dfs[i].values.tolist()
data = [col_names] + data
with open("./datasets/Processed/All_Processed.csv", "w", encoding="utf8", newline="") as f:
   w = csv.writer(f)
   w.writerows(data)

['M49_Code', 'Country_Name', 'ISO3 Code', 'Year', 'CPI_Food', 'Net_FDI', 'Life_Expectancy', 'Mean_Years_Of_Schooling', 'Prevalence_of_undernourishment']
